In [1]:
import torch

In [2]:
from comet_ml import Experiment, Optimizer

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from collections import defaultdict
from tqdm import trange
import torch
import seaborn as sns
import os
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import sklearn
import time
import datetime

plt.style.use("seaborn")
torch.set_default_dtype(torch.float32)

In [4]:
import datetime


In [5]:
import node

In [6]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, balanced_accuracy_score

In [7]:
from tabular_hypernet import HypernetworkPCA, TrainingModes, Hypernetwork
from tabular_hypernet.modules import SimpleNetwork
from tabular_hypernet.training_utils import train_slow_step, train_model, train_carthesian
from tabular_hypernet.interfaces import HypernetworkSklearnInterface, SimpleSklearnInterface
# from ipynb.fs.defs.MNIST_benchmark import test_model
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [8]:
from sklearn.metrics import classification_report, balanced_accuracy_score

In [9]:
import imblearn

In [10]:
import pyhopper

In [11]:
from loguru import logger

logger.add("log_Lymf.txt", format='{time:YYYY-MM-DD HH:mm:ss.SSS} | {message}')

1

In [12]:
import seaborn as sns

In [13]:
DEVICE="cuda:0"

In [14]:
from loguru import logger

logger.add("log.txt", format='{time:YYYY-MM-DD HH:mm:ss.SSS} | {message}')

2

In [15]:
GS_METRIC = "balanced_accuracy"

# Load data

In [16]:
DATA = "Blastchar"

In [17]:
data = pd.read_csv("./data/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [18]:
data

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [19]:
processed_data = data.copy()
del processed_data["customerID"]
processed_data = processed_data[processed_data['TotalCharges'] != " "]
processed_data['TotalCharges'] = pd.to_numeric(processed_data['TotalCharges'])

y_label_enc = LabelEncoder()
y = y_label_enc.fit_transform(processed_data["Churn"].values)
del processed_data["Churn"]

### Preprocess features

In [20]:
label_encoders = {}
for col in processed_data.columns:
    if processed_data[col].dtype == 'object':
        encoder = LabelEncoder()
        processed_data[col] = encoder.fit_transform(processed_data[col].values)
        label_encoders[col] = encoder
        
X = processed_data.values
print(X.shape)

(7032, 19)


In [21]:
X.shape

(7032, 19)

In [22]:
y.shape

(7032,)

In [23]:
n_classes = len(np.unique(y if not isinstance(y, tuple) else y_train))
n_classes

2

In [24]:
unique, counts = np.unique(y if not isinstance(y, tuple) else y_train, return_counts=True)

print(dict(zip(unique, counts)))

{0: 5163, 1: 1869}


In [25]:
n_features = X.shape[1] if not isinstance(X, tuple) else X_train.shape[1]
n_features

19

In [26]:
max_size = int(len(X)*0.7) if not isinstance(X, tuple) else len(X_train)

In [27]:
max_size = 3000

In [28]:
# print('dataset size:', len(X), '|', 'max training size:', max_size)

In [29]:
TEST_RUN = False

# Preprocess

In [30]:
np.random.seed(42)

def prepare_data(X, y, size=None):
    if isinstance(X, tuple) and isinstance(y, tuple):
        X_train, X_test = X
        y_train, y_test = y
    else:    
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=size, stratify=y)
    # X_train, y_train = imblearn.over_sampling.RandomOverSampler(random_state=42).fit_resample(X_train, y_train)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    X_train, X_test, y_train, y_test = [torch.from_numpy(x) for x in [X_train, X_test, y_train, y_test]]
    
    return X_train, X_test, y_train, y_test

In [31]:
X_train, X_test, y_train, y_test = prepare_data(X, y, size=max_size)

In [32]:
y_test.shape

torch.Size([4032])

## Benchmark

In [33]:
criterion = torch.nn.CrossEntropyLoss()

In [34]:
def _summarize_results(y_pred, y_score, y_test, labels):
    results = []
    for idx, label in enumerate(labels):
        y_pred_filt = y_pred[y_test==idx]
        y_test_filt = y_test[y_test==idx]
        acc = (y_pred_filt==y_test_filt.numpy()).sum()/len(y_test_filt)*100
        results.append({
            "Class": label,
            "Metric": acc
        })
        
    acc = (y_pred==y_test.numpy()).sum()/len(y_test)*100    
    results.append({
        "Class": "Total",
        "Metric": acc
    })
    
    
    results.append({
        "Class": "balanced_accuracy",
        "Metric": balanced_accuracy_score(y_test, torch.from_numpy(y_pred)).item()*100
    })
    
    try:
        results.append({
            "Class": "F1 score",
            "Metric": f1_score(y_test, torch.from_numpy(y_pred)).item()*100
        })
        results.append({
            "Class": "roc_auc",
            "Metric": roc_auc_score(y_test, torch.from_numpy(y_score[:, 1])).item()*100
        })
        results.append({
            "Class": "Precision",
            "Metric": precision_score(y_test, torch.from_numpy(y_pred)).item()*100
        })
        results.append({
            "Class": "Recall",
            "Metric": recall_score(y_test, torch.from_numpy(y_pred)).item()*100
        })
    except ValueError:
        pass
    return results

def test_model(model_fn, data, train_size, label_encoder=None, iters=10, as_numpy=False):
    if TEST_RUN:
        iters = 1
        
    if label_encoder is not None:
        labels = label_encoder.classes_
    else:
        labels = sorted(pd.unique(data[1][0] if isinstance(data[1], tuple) else data[1]))
    
    results = []

    for i in range(iters):
        print('iter', i+1, 'of', iters)
        X_train, X_test, y_train, y_test = prepare_data(*data, train_size)
        
        print('X_train_size', X_train.shape)
        print('y_train_size', y_train.shape)
        
        model = model_fn()

        if as_numpy:
            model.fit(X_train.numpy(), y_train.numpy());
        else:
            model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        y_score = model.predict_proba(X_test)
        results.extend(_summarize_results(y_pred, y_score, y_test, labels))
        
        del model
        torch.cuda.empty_cache()

    dframe = pd.DataFrame.from_dict(results)
#     sns.violinplot(data=dframe[dframe["Class"]!="Loss"], y="Class", x="Metric", orient='h')
    return dframe

### Param search

In [35]:
def pyhopper_best_params(model_fn, param_grid, data_size=max_size, metric=GS_METRIC, time="30m", default_params={}, n_jobs=1):
    if TEST_RUN:
        time = 30
        if 'epochs' in param_grid:
            param_grid["epochs"] = pyhopper.choice([10])
        
    logger.info(f"pyhopper_best_params {model_fn.__name__}")
    
    def objective(params):
        torch.cuda.empty_cache()

        model_results = test_model(
                        model_fn(
                            **default_params,
                            **params
                        ),
                        (X, y),
                        data_size,
                        None, 5)
        with open(f"{DATA}_{model_fn.__name__}_params.txt", "a") as f:
            f.write(str(datetime.datetime.now()) + ", " + str(params) + ", " + str(model_results[model_results["Class"]==metric]["Metric"].mean()) + "\n")
        return model_results[model_results["Class"]==metric]["Metric"].mean()

    from pyhopper.callbacks import History
    search = pyhopper.Search(param_grid)

    best_params = search.run(objective, "maximize", time, n_jobs=n_jobs, seeding_ratio=0.5)
    
    with open(f"{DATA}_{model_fn.__name__}_best_params.txt", "a") as f:
            f.write(str(best_params))
    
    print(f"{DATA}_{model_fn.__name__}_{best_params}")
    return best_params, search.history


# TRAIN MODELS

### XGBoost

In [57]:
from xgboost import XGBClassifier

In [58]:
import xgboost
xgboost.__version__

'1.6.0'

In [59]:
def get_xgboost(**params):
    random_seed = np.random.randint(1024)
    def _inner(**args):
        return XGBClassifier(
            verbosity=0,
            random_state=random_seed,
            use_label_encoder=False,
            **params,
            **args
        )
    return _inner    

#### Hyperparam tuning

In [60]:
param_grid = {
                'n_estimators': pyhopper.int(50, 3000, multiple_of=50),
                'max_depth': pyhopper.choice([2, 3, 5, 10, 15]),
                'learning_rate': pyhopper.float(1e-5,1e-1, log=True),
                'min_child_weight': pyhopper.choice([1, 2, 4, 8, 16, 32]),
                'gamma': pyhopper.choice([0, 0.001, 0.1, 1]),
             }

xgbt_best1, xgbt_history1 = pyhopper_best_params(get_xgboost, param_grid, time="60m")

2023-01-19 07:43:45.900 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params get_xgboost


  0%|          | [00:00<?]

Search is scheduled for 01:00:00 (h:m:s)
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])

iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([300

iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([300

...

/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([300

/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [61]:
param_grid = {
                'subsample': pyhopper.choice([0.5, 0.6, 0.7, 0.8, 0.9, 1]),
                'reg_lambda': pyhopper.float(1e-5, 10, init=0, log=True),
                'reg_alpha': pyhopper.float(1e-5, 10, init=0, log=True),
             }


xgbt_best2, xgbt_history2 = pyhopper_best_params(get_xgboost, param_grid, time="30m", default_params=xgbt_best1)

2023-01-19 08:43:36.609 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params get_xgboost


  0%|          | [00:00<?]

Search is scheduled for 30:00 (m:s)
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter

iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([300

/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([300

/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([300

iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



#### Best Params

'Ionosphere' {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 2, 'n_estimators': 1500}


In [62]:
xgboost_best = {**xgbt_best1, **xgbt_best2}

In [64]:
xgboost_best

{'n_estimators': 400,
 'max_depth': 2,
 'learning_rate': 1.1265409543614521e-05,
 'min_child_weight': 16,
 'gamma': 0.1,
 'subsample': 1,
 'reg_lambda': 5.619256074268084e-05,
 'reg_alpha': 1e-05}

In [69]:
data_size = max_size

xgb_dframe = test_model(get_xgboost(**xgboost_best),
                        (X, y),
                        data_size,
                        label_encoder=None, iters=10)

iter 1 of 10
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 10
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 3 of 10
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 4 of 10
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 5 of 10
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 6 of 10
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 7 of 10
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 8 of 10
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 9 of 10
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 10 of 10
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])


In [70]:
res = xgb_dframe[xgb_dframe["Class"]=="balanced_accuracy"].reset_index(drop=True)["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f} (max: {res.max():.2f})")

3000: 71.74 ~ 1.33 (max: 73.92)


In [71]:
xgb_dframe.groupby(['Class']).mean()

,Metric
Class,
0,77.337838
1,66.147388
F1 score,57.780351
Precision,51.473977
Recall,66.147388
Total,74.362599
balanced_accuracy,71.742613
roc_auc,77.995746


### NODE

In [36]:
from qhoptim.pyt import QHAdam

In [37]:
def node_fn(layer_dim=128, num_layers=1, depth=3):
    def _inner():
        network = torch.nn.Sequential(
            node.DenseBlock(X_train.shape[1], 
                            layer_dim=layer_dim,
                            num_layers=num_layers, 
                            tree_dim=n_classes+1, 
                            depth=depth, 
                            flatten_output=False,
                            choice_function=node.entmax15, 
                            bin_function=node.entmoid15
                           ),
            node.Lambda(lambda x: x.mean(dim=1))
        )
        
        
        network = network.to(DEVICE)
        network.device=DEVICE
        
        with torch.no_grad():
            res = network(torch.as_tensor(X_train, device=DEVICE).to(torch.float32))
        
            
        optimizer_params = { 'nus':(0.7, 1.0), 'betas':(0.95, 0.998) }
        optim = QHAdam(network.parameters(), **optimizer_params)
            
        network = SimpleSklearnInterface(network, device=DEVICE, epochs=150, batch_size=32)
        network.optimizer = optim
        return network
    return _inner

#### Tune hyperparams

In [38]:
# 'layer_dim': hp.quniform('layer_dim', 100, 1200, 100),
# 'num_layers': hp.quniform('num_layers', 1, 4, 1),
# 'depth': hp.quniform('depth', 2, 7, 1)
                    
param_grid = {
    'layer_dim': pyhopper.int(64, 1024, power_of=2),
    'num_layers': pyhopper.int(1, 5),
    'depth': pyhopper.int(2, 7),
}

node_best, node_history = pyhopper_best_params(node_fn, param_grid, time=f"{15}m", n_jobs=1)
node_best

2023-01-18 22:47:23.779 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params node_fn


  0%|          | [00:00<?]

/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:72: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(torch.float32)


Search is scheduled for 15:00 (m:s)
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/qhoptim/pyt/qhadam.py:133: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)
  exp_avg.mul_(beta1_adj).add_(1.0 - beta1_adj, d_p)


CTRL+C received. Will terminate once the currently running candidates finished
CTRL+C received. Will terminate once the currently running candidates finished
Will force termination on 2/3 signals
Terminate


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd5fbe129d0>
Traceback (most recent call last):
  File "/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1322, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/multiprocessing/connection.py", line 936, in wait
    ready = selector.select(timeout)
  File "/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/

Terminate
Remote process caught exception in objective function: 
Traceback (most recent call last):
  File "/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/pyhopper/parallel.py", line 223, in execute
    iter_or_result = objective_function(candidate, **kwargs)
  File "/tmp/ipykernel_3155690/567169155.py", line 12, in objective
    model_results = test_model(
  File "/tmp/ipykernel_3155690/3937386562.py", line 68, in test_model
    model.fit(X_train, y_train)
  File "/home/z1157095/hypernet-cnn/hypernet/tabular_hypernet/interfaces.py", line 21, in fit
    basic_train_loop(self.network, self.optimizer, self.criterion, train_data, self.epochs, self.device)
  File "/home/z1157095/hypernet-cnn/hypernet/tabular_hypernet/training_utils.py", line 320, in basic_train_loop
    loss.backward()
  File "/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/torch/_tensor.py", line 363, in backward
    torch.autograd.backward(self, gradient, retain_graph, create

ValueError: Pyhopper - Remote process caught exception

#### Use best hyperparams

In [40]:
%%capture --no-stdout

size=max_size

node_results = test_model(node_fn(**node_best),
                    (X, y),
                    size,
                    label_encoder=None, iters=10)
res = node_results[node_results["Class"]=="balanced_accuracy"].reset_index(drop=True)["Metric"]
print(f"{size}: {res.mean():.2f} ~ {res.std():.2f}, (max: {res.max():.2f})")

iter 1 of 10
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
iter 2 of 10
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])


KeyboardInterrupt: 

In [40]:
res = node_results[node_results["Class"]=="balanced_accuracy"].reset_index(drop=True)["Metric"]
print(f"{size}: {res.mean():.2f} ~ {res.std():.2f}, (max: {res.max():.2f})")

207: 82.38 ~ 4.59, (max: 88.39)


In [50]:
node_results[node_results["Class"]=="balanced_accuracy"].mean()

/tmp/ipykernel_966419/3426196394.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  node_results[node_results["Class"]=="balanced_accuracy"].mean()


Metric    82.380952
dtype: float64

In [46]:
node_results.groupby(['Class']).mean()

,Metric
Class,
0,88.333333
1,76.428571
F1 score,80.404076
Precision,85.716200
Recall,76.428571
Total,82.777778
balanced_accuracy,82.380952
roc_auc,91.225198


In [47]:
node_results.groupby(['Class']).std()

,Metric
Class,
0,7.095578
1,8.798434
F1 score,5.737790
Precision,6.292452
Recall,8.798434
Total,4.513355
balanced_accuracy,4.591550
roc_auc,2.024427


### Dropout Neural network

In [59]:
def network_fn1(epochs=100, drop1=0.3, drop2=0.5, batch_size=32, lr=3e-4):
    
    def _inner():
        network = torch.nn.Sequential(
                        torch.nn.Dropout(drop1),
                        torch.nn.Linear(n_features, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop2),
                        torch.nn.Linear(64, n_classes)
                    ).to(DEVICE).train()

        network = SimpleSklearnInterface(network, epochs=epochs, batch_size=batch_size, lr=lr)
        return network
    return _inner



def network_fn2(epochs=100, drop1=0.3, drop2=0.5, drop3=0.5, batch_size=32, lr=3e-4):
    
    def _inner():
        network = torch.nn.Sequential(
                        torch.nn.Dropout(drop1),
                        torch.nn.Linear(n_features, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop2),
                        torch.nn.Linear(64, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop3),
                        torch.nn.Linear(64, n_classes)
                    ).to(DEVICE).train()

        network = SimpleSklearnInterface(network, epochs=epochs, batch_size=batch_size, lr=lr)
        return network
    return _inner



def network_fn3(epochs=100, drop1=0.3, drop2=0.5, drop3=0.5, drop4=0.5, batch_size=32, lr=3e-4):
    
    def _inner():
        network = torch.nn.Sequential(
                        torch.nn.Dropout(drop1),
                        torch.nn.Linear(n_features, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop2),
                        torch.nn.Linear(64, 128),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop3),
                        torch.nn.Linear(128, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop4),
                        torch.nn.Linear(64, n_classes)
                    ).to(DEVICE).train()

        network = SimpleSklearnInterface(network, epochs=epochs, batch_size=batch_size, lr=lr)
        return network
    return _inner

#### Find Hyperparams

### Dropout 1 layer

In [46]:
param_grid = {
                "epochs": pyhopper.choice([100, 150]),
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1]),
                "batch_size": pyhopper.choice([32, 64]),
             }

nn_fn1_best_params, nn_fn1_history = pyhopper_best_params(network_fn1, param_grid, time="60m")
nn_fn1_best_params

2022-12-13 16:12:27.803 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_fn1


  0%|          | [00:00<?]

Search is scheduled for 01:00:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5

{'epochs': 100, 'drop1': 0.5, 'drop2': 0.7, 'lr': 0.0003, 'batch_size': 32}

In [47]:
data_size = max_size

nn1_results = test_model(network_fn1(**nn_fn1_best_params),
                (X, y),
                data_size,
                None, 10)

iter 1 of 10
iter 2 of 10
iter 3 of 10
iter 4 of 10
iter 5 of 10
iter 6 of 10
iter 7 of 10
iter 8 of 10
iter 9 of 10
iter 10 of 10


In [48]:
res = nn1_results[nn1_results["Class"]=="Balanced Acc score"]["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f} (max: {res.max():.2f})")

756: nan ~ nan (max: nan)


### Dropout 2 layers

In [49]:
param_grid = {
                "epochs": pyhopper.choice([100, 150]),
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop3": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1]),
                "batch_size": pyhopper.choice([32, 64]),
             }

nn_fn2_best_params, nn_fn2_history = pyhopper_best_params(network_fn2, param_grid, time="70m")
nn_fn2_best_params

2022-12-13 17:17:56.668 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_fn2


  0%|          | [00:00<?]

Search is scheduled for 01:10:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5

{'epochs': 100,
 'drop1': 0.5,
 'drop2': 0.1,
 'drop3': 0.7,
 'lr': 0.0003,
 'batch_size': 64}

In [49]:
data_size = max_size

nn2_results = test_model(network_fn2(**nn_fn2_best_params),
                (X, y),
                data_size,
                None, 10)

iter 1 of 10
iter 2 of 10
iter 3 of 10
iter 4 of 10
iter 5 of 10
iter 6 of 10
iter 7 of 10
iter 8 of 10
iter 9 of 10
iter 10 of 10


### Dropout 3 layers

In [78]:
param_grid = {
                "epochs": pyhopper.choice([100, 150]),
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop3": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop4": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1]),
                "batch_size": pyhopper.choice([32, 64]),
             }

nn_fn3_best_params, nn_fn3_history = pyhopper_best_params(network_fn3, param_grid, time="90m")
nn_fn3_best_params

2023-01-18 17:19:44.938 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_fn3


  0%|          | [00:00<?]

Search is scheduled for 01:30:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 3 of 5


...

/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 1 of 5


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
====================== Summary ======================
Mode              : Best f : Steps : Time            
----------------  : ----   : ----  : ----            
Initial solution  : 72.51  : 1     : 04:01 (m:s)     
Random seeding    : 72.42  : 10    : 41:31 (m:s)     
Local sampling    : 72.64  : 10    : 45:59 (m:s)     
Duplicates        : -      : 4     : -               
----------------  : ----   : ----  : ----            
Total             : 72.64  : 25    : 01:31:31 (h:m:s)
Blastchar_network_fn3_{'epochs': 100, 'drop1': 0.3, 'drop2': 0.3, 'drop3': 0.3, 'drop4': 0.3, 'lr': 3e-05, 'batch_size': 64}


{'epochs': 100,
 'drop1': 0.3,
 'drop2': 0.3,
 'drop3': 0.3,
 'drop4': 0.3,
 'lr': 3e-05,
 'batch_size': 64}

In [79]:
nn_fn3_best_params

{'epochs': 100,
 'drop1': 0.3,
 'drop2': 0.3,
 'drop3': 0.3,
 'drop4': 0.3,
 'lr': 3e-05,
 'batch_size': 64}

In [85]:
max_size

3000

In [89]:
data_size = max_size

nn3_results = test_model(network_fn3(**nn_fn3_best_params),
                (X, y),
                data_size,
                None, 10)

iter 1 of 10
data_size torch.Size([3000, 19])
iter 2 of 10
data_size torch.Size([3000, 19])
iter 3 of 10
data_size torch.Size([3000, 19])


KeyboardInterrupt: 

In [90]:
nn3_results

,Class,Metric
0,0,86.858108
1,1,59.235075
2,Total,79.513889
3,balanced_accuracy,73.046591
4,F1 score,60.591603
...,...,...
75,balanced_accuracy,72.039885
76,F1 score,59.244728
77,roc_auc,83.373525
78,Precision,62.461220


In [93]:
nn3_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])

,mean,std,max
Class,,,
0,86.293919,1.046516,87.736486
1,58.339552,2.394965,61.660448
F1 score,59.456400,1.189401,61.459786
Precision,60.692145,1.219709,62.461220
Recall,58.339552,2.394965,61.660448
Total,78.861607,0.488821,79.513889
balanced_accuracy,72.316736,0.859346,73.769413
roc_auc,83.182302,0.460967,83.655834


In [84]:
res = nn3_results[nn3_results["Class"]=="balanced_accuracy"].reset_index(drop=True)["Metric"]
print(f"{DATA}: {res.mean():.2f} ~ {res.std():.2f}, (max: {res.max():.2f})")

Blastchar: 72.32 ~ 0.86, (max: 73.77)


### HypernetworkPCA

In [81]:
def network_pca_fn(epochs=100, masks_no=100, mask_size=100, target_size=100, n_comp=5, lr=3e-4, batch_size=64, verbose=False):
    def _inner():
        hypernet = HypernetworkPCA(
                        target_architecture=[(mask_size, target_size), (target_size, n_classes)], 
                        test_nodes=masks_no,
                        architecture=torch.nn.Sequential(torch.nn.Linear(n_comp, 64), 
                            torch.nn.ReLU(),
                            torch.nn.Linear(64, 128),
                            torch.nn.ReLU(),
                            torch.nn.Dropout(),
                            torch.nn.Linear(128, 128),
                            torch.nn.ReLU(),
                        ),
                        mode=TrainingModes.CARTHESIAN,
                        input_size=n_features
                    ).to(DEVICE)    
        hypernet = hypernet.train()

        network = HypernetworkSklearnInterface(hypernet, device=DEVICE, epochs=epochs, batch_size=batch_size, verbose=verbose, lr=lr)
        return network
    return _inner

In [54]:
def figure_to_html():
    tmpfile = BytesIO()
    plt.gcf().savefig(tmpfile, format='png')
    encoded = base64.b64encode(tmpfile.getvalue()).decode('utf-8')

    html = '<img src=\'data:image/png;base64,{}\'>'.format(encoded)
    return html

#### Find hyperparams

In [55]:
param_grid = {
    "epochs": pyhopper.choice([100]),
    "masks_no": pyhopper.choice([100, 150, 200, 300, 500]),#pyhopper.int(50, 200, multiple_of=10),
    "mask_size": pyhopper.choice([5, 10, 20, 50, 100, 200, 400]),
    "target_size": pyhopper.choice([5, 10, 20, 50]),
    "n_comp": pyhopper.choice([3, 10, 25, 50, 80]),
    "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1]),
    "batch_size": pyhopper.choice([32, 64]),

}

hp_pca_best_params, hp_pca_history = pyhopper_best_params(network_pca_fn, param_grid, time="480m")
hp_pca_best_params

2022-12-13 19:56:50.812 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_pca_fn


  0%|          | [00:00<?]

Search is scheduled for 08:00:00 (h:m:s)
iter 1 of 5
torch.Size([1, 128])
855
iter 2 of 5
torch.Size([1, 128])
855
iter 3 of 5
torch.Size([1, 128])
855
iter 4 of 5
torch.Size([1, 128])
855
iter 5 of 5
torch.Size([1, 128])
855
iter 1 of 5
torch.Size([1, 128])
855
iter 2 of 5
torch.Size([1, 128])
855
iter 3 of 5
torch.Size([1, 128])
855
iter 4 of 5
torch.Size([1, 128])
855
iter 5 of 5
torch.Size([1, 128])
855
iter 1 of 5
torch.Size([1, 128])
855
iter 2 of 5
torch.Size([1, 128])
855
iter 3 of 5
torch.Size([1, 128])
855
iter 4 of 5
torch.Size([1, 128])
855
iter 5 of 5
torch.Size([1, 128])
855
iter 1 of 5
torch.Size([1, 128])
855
iter 2 of 5
torch.Size([1, 128])
855
iter 3 of 5
torch.Size([1, 128])
855
iter 4 of 5
torch.Size([1, 128])
855
iter 5 of 5
torch.Size([1, 128])
855
iter 1 of 5
torch.Size([1, 128])
855
iter 2 of 5
torch.Size([1, 128])
855
iter 3 of 5
torch.Size([1, 128])
855
iter 4 of 5
torch.Size([1, 128])
855
iter 5 of 5
torch.Size([1, 128])
855
iter 1 of 5
torch.Size([1, 128])
8

{'epochs': 100,
 'masks_no': 200,
 'mask_size': 200,
 'target_size': 50,
 'n_comp': 3,
 'lr': 3e-05,
 'batch_size': 32}

In [85]:
hp_pca_best_params

{'epochs': 100,
 'masks_no': 50,
 'mask_size': 400,
 'target_size': 50,
 'n_comp': 3,
 'lr': 3e-05,
 'batch_size': 32}

'Libras'
{'epochs': 150,
 'masks_no': 70,
 'mask_size': 20,
 'target_size': 10,
 'n_comp': 10}

'Lymphography' {'epochs': 120, 'masks_no': 50, 'mask_size': 4, 'target_size': 20, 'n_comp': 6}

In [ ]:
DATA

Ionosphere
{'epochs': 100, 'masks_no': 60, 'mask_size': 5, 'target_size': 10}


In [82]:
epochs = 100#hp_pca_best_params['epochs']
masks_no = hp_pca_best_params['masks_no']
mask_size = hp_pca_best_params['mask_size']
target_size = hp_pca_best_params['target_size']
n_comp = hp_pca_best_params['n_comp']
lr = hp_pca_best_params['lr']
batch_size = hp_pca_best_params['batch_size']
data_size = max_size

nn_pca_results = test_model(network_pca_fn(target_size=target_size, mask_size=mask_size, masks_no=masks_no, n_comp=n_comp, epochs=epochs, verbose=True),
                (X, y),
                data_size,
                None, 10)

# exp.log_table("metrics.csv", nn_pca_results.groupby("Class").mean())
# exp.log_metric("f1_score", nn_pca_results.groupby("Class").mean().loc["F1 score"])

iter 1 of 10
torch.Size([1, 128])
855


100%|██████████| 100/100 [02:50<00:00,  1.71s/it]


iter 2 of 10
torch.Size([1, 128])
855


100%|██████████| 100/100 [02:49<00:00,  1.70s/it]


iter 3 of 10
torch.Size([1, 128])
855


...

100%|██████████| 100/100 [02:49<00:00,  1.70s/it]


iter 9 of 10
torch.Size([1, 128])
855


100%|██████████| 100/100 [02:50<00:00,  1.70s/it]


iter 10 of 10
torch.Size([1, 128])
855


100%|██████████| 100/100 [02:50<00:00,  1.70s/it]


In [84]:
res = nn_pca_results[nn_pca_results["Class"]=="Total"].reset_index(drop=True)["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f} (max: {res.max():.2f})")

756: 94.97 ~ 0.97 (max: 96.60)


### Hypernetwork

In [51]:
def network_hp_fn(epochs=150, masks_no=100, mask_size=100, target_size=100, lr=3e-4, batch_size=64, verbose=False):
    def _inner():
        hypernet = Hypernetwork(
                        target_architecture=[(mask_size, target_size), (target_size, n_classes)],
                        test_nodes=masks_no,
                        architecture=torch.nn.Sequential(torch.nn.Linear(n_features, 64), 
                            torch.nn.ReLU(),
                            torch.nn.Linear(64, 128),
                            torch.nn.ReLU(),
                            torch.nn.Dropout(),
                            torch.nn.Linear(128, 128),
                            torch.nn.ReLU(),
                        ),
                        mode=TrainingModes.CARTHESIAN,
                    ).to(DEVICE)    
        hypernet = hypernet.train()

        network = HypernetworkSklearnInterface(hypernet, device=DEVICE, epochs=epochs, batch_size=batch_size, verbose=verbose, lr=lr)
        return network
    return _inner


#### Find hyperparams

In [52]:
param_grid = {
    "epochs": pyhopper.choice([100]),
    "masks_no": pyhopper.choice([3, 5, 10, 50, 75, 100, 150, 200, 300, 500]), #pyhopper.int(10, 400, multiple_of=10),
    "mask_size": pyhopper.choice([3, 5, 7, 10, 13, 17]),
    "target_size": pyhopper.choice([5, 10, 20, 50]),
    "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1]),
    "batch_size": pyhopper.choice([128]),
}

hp_best_params, hp_history = pyhopper_best_params(network_hp_fn, param_grid, time=f"{6*60}m")
hp_best_params

2023-01-19 00:56:44.235 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params network_hp_fn


  0%|          | [00:00<?]

Search is scheduled for 06:00:00 (h:m:s)
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
torch.Size([1, 128])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
torch.Size([1, 128])


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
torch.Size([1, 128])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
torch.Size([1, 128])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
torch.Size([1, 128])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
torch.Size([1, 128])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
torch.Size([1, 128])
iter 3 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
torch.Size([1, 128])
iter 4 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
torch.Size([1, 128])
iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
torch.Size([1, 128])
iter 1 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
torch.Size([1, 128])
iter 2 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
torch.Size(

...

iter 5 of 5
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
torch.Size([1, 128])


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


====================== Summary ======================
Mode              : Best f : Steps : Time            
----------------  : ----   : ----  : ----            
Initial solution  : 55.79  : 1     : 04:36 (m:s)     
Random seeding    : 59.73  : 2     : 05:08:07 (h:m:s)
----------------  : ----   : ----  : ----            
Total             : 59.73  : 3     : 05:12:43 (h:m:s)
Blastchar_network_hp_fn_{'epochs': 100, 'masks_no': 200, 'mask_size': 7, 'target_size': 20, 'lr': 0.003, 'batch_size': 128}


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'epochs': 100,
 'masks_no': 200,
 'mask_size': 7,
 'target_size': 20,
 'lr': 0.003,
 'batch_size': 128}

In [72]:
hp_best_params

{'epochs': 100,
 'masks_no': 200,
 'mask_size': 7,
 'target_size': 20,
 'lr': 0.003,
 'batch_size': 128}

#### Train using the best hyperparams

In [73]:
epochs = 100#hp_best_params['epochs']
masks_no = 150#hp_best_params['masks_no']
mask_size = 7#hp_best_params['mask_size']
target_size = 20#hp_best_params['target_size']
data_size = max_size
batch_size = 128#hp_best_params['batch_size']
lr = 3e-04#hp_best_params['lr']


hyper_results = test_model(network_hp_fn(epochs, masks_no, mask_size, target_size, batch_size=batch_size,verbose=True),
                    (X, y),
                    data_size,
                    None, 10)

iter 1 of 10
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
torch.Size([1, 128])


100%|██████████| 100/100 [13:48<00:00,  8.29s/it]


iter 2 of 10
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
torch.Size([1, 128])


100%|██████████| 100/100 [13:53<00:00,  8.33s/it]


iter 3 of 10
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
torch.Size([1, 128])


...

100%|██████████| 100/100 [13:43<00:00,  8.24s/it]


iter 9 of 10
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
torch.Size([1, 128])


100%|██████████| 100/100 [13:47<00:00,  8.28s/it]


iter 10 of 10
X_train_size torch.Size([3000, 19])
y_train_size torch.Size([3000])
torch.Size([1, 128])


100%|██████████| 100/100 [13:40<00:00,  8.21s/it]


In [74]:
hyper_results[hyper_results["Class"]=="balanced_accuracy"].mean()

/tmp/ipykernel_3155690/3655004452.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  hyper_results[hyper_results["Class"]=="balanced_accuracy"].mean()


Metric    69.819534
dtype: float64

In [56]:
res = hyper_results[hyper_results["Class"]=="balanced_accuracy"].reset_index(drop=True)["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f}, (max: {res.max():.2f})")

3000: 60.10 ~ 5.11, (max: 66.24)


### Random forest

In [68]:
from sklearn.ensemble import RandomForestClassifier

In [69]:
def get_rf(**params):
    random_seed = np.random.randint(1024)
    def _inner():
        return RandomForestClassifier(
            random_state=random_seed,
            **params
        )
    return _inner

#### Find hyperparams

In [70]:
param_grid = {
                'n_estimators': pyhopper.int(50, 3000, multiple_of=50),
                'max_features': pyhopper.choice([None, 'sqrt', 0.2, 0.3, 0.5, 0.7]),
                'criterion' : pyhopper.choice(['gini', 'entropy']),
                'max_depth': pyhopper.choice([None, 2, 4, 8, 16]),
             }

rf_best, rf_history = pyhopper_best_params(get_rf, param_grid, time="90m")
rf_best

2023-01-18 12:57:03.101 | INFO     | __main__:pyhopper_best_params:7 - pyhopper_best_params get_rf


  0%|          | [00:00<?]

Search is scheduled for 01:30:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5

iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter

iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter

iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
====================== Summary ======================
Mode              : Best f : Steps : Time            
----------------  : ----   : ----  : ----            
Initial solution  : 82.05  : 1     : 11.11 s         
Random seeding    : 86.01  : 258   : 44:55 (m:s)     
Local sampling    : 86.67  : 159   : 44:41 (m:s)     
Duplicates        : -      : 182   : -               
----------------  : ----   : ----  : ----            
Total             : 86.67  : 600   : 01:29:48 (h:m:s

{'n_estimators': 3000,
 'max_features': 'sqrt',
 'criterion': 'entropy',
 'max_depth': None}

#### Use best params

In [71]:
size = max_size

rf_dframe = test_model(get_rf(**rf_best), 
                        (X, y),
                        size,
                        None, iters=10)

iter 1 of 10
iter 2 of 10
iter 3 of 10
iter 4 of 10
iter 5 of 10
iter 6 of 10
iter 7 of 10
iter 8 of 10
iter 9 of 10
iter 10 of 10


In [73]:
rf_dframe[rf_dframe["Class"]=="balanced_accuracy"].mean()

/tmp/ipykernel_966419/2964448691.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  rf_dframe[rf_dframe["Class"]=="balanced_accuracy"].mean()


Metric    82.440476
dtype: float64

In [72]:
res = rf_dframe[rf_dframe["Class"]=="balanced_accuracy"].reset_index(drop=True)["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f}, (max: {res.max():.2f})")

207: 82.44 ~ 3.30, (max: 88.84)


# Collect analytics

In [98]:
d = {}

In [99]:
d['Random forest'] = rf_dframe.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Hypernet'] = hyper_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['HypernetPCA'] = nn_pca_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Dropout_1'] = nn1_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Dropout_2'] = nn2_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Dropout_3'] = nn3_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Node'] = node_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['XGBoost'] = xgb_dframe.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])

In [100]:
all_models_df=pd.concat(d, axis=0)
all_models_df

mean       std         max
              Class                                              
Random forest 0                   97.777778  2.868877  100.000000
              1                   93.055556  3.982558  100.000000
              2                   95.000000  3.657114  100.000000
              3                   84.722222  4.192308   88.888889
              4                  100.000000  0.000000  100.000000
...                                     ...       ...         ...
XGBoost       6                   91.388889  4.233011   97.222222
              7                   96.666667  3.414646  100.000000
              8                   77.777778  7.407407   91.666667
              Total               90.493827  2.051431   94.444444
              balanced_accuracy   90.493827  2.051431   94.444444

[88 rows x 3 columns]

In [101]:
os.environ['COMET_KEY']

'UXrV5UxyhTK3cyQNG6BDuc4bE'

In [102]:
all_models_df.to_csv(f"{DATA}_metrics.csv")

In [103]:
exp = Experiment(os.environ.get("COMET_KEY"), 'hypernet-uci-tune')
# exp.log_parameters({"epochs": epochs, "mask_size": mask_size, "masks_no": masks_no, "data_size": data_size})
exp.add_tag(f"hypernet-tune2{DATA}")
exp.log_table(f"{DATA}_metrics.csv", all_models_df)

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/abulenok/hypernet-uci-tune/cd3d48b8f7b048d1934b06562ea42da7
COMET INFO:   Parameters:
COMET INFO:     copy                       : True
COMET INFO:     iterated_power             : auto
COMET INFO:     n_components               : 3
COMET INFO:     n_oversamples              : 10
COMET INFO:     power_iteration_normalizer : auto
COMET INFO:     random_state               : 1
COMET INFO:     svd_solver                 : auto
COMET INFO:     tol                        : 0.0
COMET INFO:     whiten                     : Fal

{'web': 'https://www.comet.com/api/asset/download?assetId=96b7c9337313423fafa70e413639f6d6&experimentKey=4de718e831d54a9a9d63fd69a09f30c0',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=96b7c9337313423fafa70e413639f6d6&experimentKey=4de718e831d54a9a9d63fd69a09f30c0',
 'assetId': '96b7c9337313423fafa70e413639f6d6'}

### Replace some data in existing

In [104]:
# tmp_df = pd.concat(d, axis=0)
# tmp_df = tmp_df.reset_index()
# tmp_df = tmp_df.rename(columns={tmp_df.columns[0]: DATA})

# tmp_df

In [105]:
# all_models_df = pd.read_csv(f"{DATA}_metrics.csv")
# all_models_df = all_models_df.rename(columns={all_models_df.columns[0]: DATA})
# all_models_df = all_models_df.drop(all_models_df[all_models_df.iloc[:, 0] == 'Hypernet'].index)
# all_models_df = all_models_df.drop(all_models_df[all_models_df.iloc[:, 0] == 'HypernetPCA'].index)

# all_models_df

In [106]:
# all_models_df = pd.concat([all_models_df, tmp_df])
# all_models_df

In [107]:
f"{DATA}_metrics.csv"

'cnae_metrics.csv'

In [108]:
all_models_df = pd.read_csv(f"{DATA}_metrics.csv")

In [109]:
all_models_df.reset_index()

,index,Unnamed: 0,Class,mean,std,max
0,0,Random forest,0,97.777778,2.868877,100.000000
1,1,Random forest,1,93.055556,3.982558,100.000000
2,2,Random forest,2,95.000000,3.657114,100.000000
3,3,Random forest,3,84.722222,4.192308,88.888889
4,4,Random forest,4,100.000000,0.000000,100.000000
...,...,...,...,...,...,...
83,83,XGBoost,6,91.388889,4.233011,97.222222
84,84,XGBoost,7,96.666667,3.414646,100.000000
85,85,XGBoost,8,77.777778,7.407407,91.666667
86,86,XGBoost,Total,90.493827,2.051431,94.444444


In [110]:
# all_models_df = all_models_df.drop(all_models_df.columns[0], axis=1)
# all_models_df

In [111]:
all_models_df[all_models_df['Class'] == 'balanced_accuracy']

,Unnamed: 0,Class,mean,std,max
10,Random forest,balanced_accuracy,91.851852,1.358648,94.135802
21,Hypernet,balanced_accuracy,92.253086,2.551150,95.370370
32,HypernetPCA,balanced_accuracy,94.969136,0.965654,96.604938
43,Dropout_1,balanced_accuracy,94.537037,1.029321,95.987654
54,Dropout_2,balanced_accuracy,94.012346,0.685528,95.370370
65,Dropout_3,balanced_accuracy,94.969136,0.770575,96.296296
76,Node,balanced_accuracy,94.722222,1.168047,97.222222
87,XGBoost,balanced_accuracy,90.493827,2.051431,94.444444


In [112]:
tmp = all_models_df

In [113]:
tmp = tmp.rename(columns={tmp.columns[0]: DATA})

In [114]:
tmp

,cnae,Class,mean,std,max
0,Random forest,0,97.777778,2.868877,100.000000
1,Random forest,1,93.055556,3.982558,100.000000
2,Random forest,2,95.000000,3.657114,100.000000
3,Random forest,3,84.722222,4.192308,88.888889
4,Random forest,4,100.000000,0.000000,100.000000
...,...,...,...,...,...
83,XGBoost,6,91.388889,4.233011,97.222222
84,XGBoost,7,96.666667,3.414646,100.000000
85,XGBoost,8,77.777778,7.407407,91.666667
86,XGBoost,Total,90.493827,2.051431,94.444444


In [115]:
tmp[tmp['Class'] == "balanced_accuracy"]

,cnae,Class,mean,std,max
10,Random forest,balanced_accuracy,91.851852,1.358648,94.135802
21,Hypernet,balanced_accuracy,92.253086,2.551150,95.370370
32,HypernetPCA,balanced_accuracy,94.969136,0.965654,96.604938
43,Dropout_1,balanced_accuracy,94.537037,1.029321,95.987654
54,Dropout_2,balanced_accuracy,94.012346,0.685528,95.370370
65,Dropout_3,balanced_accuracy,94.969136,0.770575,96.296296
76,Node,balanced_accuracy,94.722222,1.168047,97.222222
87,XGBoost,balanced_accuracy,90.493827,2.051431,94.444444
